In [1]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



In [2]:

#%%Read images

images=[cv2.imread(file) for file in glob.glob("F:/pdf e-reading/Mtech/2nd semester/Machine Learning/Lab/Final_lab_assignment/Images/*.bmp")]
backgroundtruth=[cv2.imread(gndtruth,0) for gndtruth in glob.glob("F:/pdf e-reading/Mtech/2nd semester/Machine Learning/Lab/Final_lab_assignment/BakSubGroundTruth-20200603T191438Z-001/BakSubGroundTruth/*.bmp")]
print(len(images))
print(len(backgroundtruth))

505
501


In [3]:
##Foreground detection
def get_Ct(Dimg,lmbda):
#     img1=imgg**2<=((lmbda**2)*9)
#     img1=img1*255
    Ct= np.zeros((Dimg.shape[0],Dimg.shape[1]))
    thresh=(lmbda**2)*9
    for i in range(Dimg.shape[0]):
        for j in range(Dimg.shape[1]):
            if(Dimg[i,j,0]**2<=thresh and Dimg[i,j,1]<=thresh and Dimg[i,j,2]<=thresh):
                Ct[i][j]=255
            else:
                Ct[i][j]=0
    
    return Ct
# img1.shape
# plt.imshow(img1*255)

## Noise Removal


Noise removal will be done using neighbourhood voting.

In [4]:
##Noise removal
def remove_noise(image):
    """
    Removes noise from a 2D image.
    Takes 2D array(image) as input.
    Returns : Noise removed 2d array .
    """
    threshold=0.3
    image=cv2.copyMakeBorder(image,2,2,2,2,borderType=cv2.BORDER_CONSTANT)
    ## Make a 3d kernel
    
    m=5  #window size
    window=np.ones((m,m),np.float32)/(m*m)

    img2=np.zeros((image.shape[0],image.shape[1]),dtype='uint8')
    ##image_slicing
    count=0
    for i in range(2,image.shape[0]-2):
        for j in range(2,image.shape[1]-2):
            slice_=image[i-2:i+3,j-2:j+3]
            conv= (slice_ * window ).sum()
            if((conv/25) >= threshold):
                img2[i][j]=0
            else:
                img2[i][j]=255
    return img2



# Background Model Update


In [25]:
def alpha(t):
    tc=0.01
    
    if(t<=tc):
        return 1/t
    else:
        return 1/tc

def kron_delta(pixel):
    if(pixel==0):
        return 1
    else:
        return 0
    
vect_kdel = np.vectorize(kron_delta)    
    

# iim
def conv_vC_to_3D_matrix(vC):                                           # for easier background model update   
    vC_3d = np.zeros((images[0].shape), dtype = np.int32)
    for i in range(images[0].shape[0]):
        for j in range(images[0].shape[1]):
            if vC[i][j] == 255:
                vC_3d[i][j] = [255, 255, 255]
    return vC_3d



In [29]:
Vd=9
curr_bckgrnd=np.zeros((images[0].shape[0],images[0].shape[1],images[0].shape[2]))
curr_V=np.zeros((images[0].shape[0],images[0].shape[1],images[0].shape[2]))
prev_bckgrnd=images[0]
prev_V=Vd*np.ones((images[0].shape[0],images[0].shape[1],images[0].shape[2]))


TPR=[]
FPR=[]
lmbdas=[0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
for lmbda in lmbdas:
    Y_pred=[]
    for i in range(1, len(images)-3):
        print("Img :", i)
        Dimg=images[i]-images[i-1]  ## Difference of images
        Ct=get_Ct(Dimg,lmbda)
        ## Noise removal
        vCt=remove_noise(Ct)
        
#         vCt=conv_vC_to_3D_matrix(vCt)
        Y_pred.append(vCt)

        ##Background Update
        for x in range(images[0].shape[0]):
            for y in range(images[0].shape[1]):
                curr_bckgrnd[x][y][:]=prev_bckgrnd[x][y]+ kron_delta(vCt[x][y]) * alpha(i) * Dimg[x][y]
                curr_V[x][y][:] =kron_delta(Ct[x][y]-255) * prev_V[x][y] + kron_delta(vCt[x][y])* (1-alpha(i)*(prev_V[x][y] + (alpha(i) *Dimg[x][y]**2)))
        
#         curr_bckgrnd=prev_bckgrnd +  vect_kdel(vCt)*alpha(i) * Dimg
#         curr_V =kron_delta(Ct-255) * prev_V + vect_kdel(vCt)* (1-alpha(i)*(prev_V + (alpha(i) *Dimg**2)))
        
        prev_bckgrnd=curr_bckgrnd
        prev_V=curr_V
    
    TN,FP,FN,TP = confusion_matrix(backgroundtruth,Y_pred)
    TPR.append(TP/(TP+FN))
    FPR.append(FP/(FP+TN))
    

Img : 1
Img : 2
Img : 3
Img : 4
Img : 5
Img : 6
Img : 7
Img : 8
Img : 9
Img : 10
Img : 11
Img : 12
Img : 13
Img : 14
Img : 15
Img : 16
Img : 17
Img : 18
Img : 19
Img : 20
Img : 21
Img : 22
Img : 23
Img : 24
Img : 25
Img : 26
Img : 27
Img : 28
Img : 29
Img : 30
Img : 31
Img : 32
Img : 33
Img : 34
Img : 35
Img : 36
Img : 37
Img : 38
Img : 39
Img : 40
Img : 41
Img : 42
Img : 43
Img : 44
Img : 45
Img : 46
Img : 47
Img : 48
Img : 49
Img : 50
Img : 51
Img : 52
Img : 53
Img : 54
Img : 55
Img : 56
Img : 57
Img : 58
Img : 59
Img : 60
Img : 61
Img : 62
Img : 63
Img : 64
Img : 65
Img : 66
Img : 67
Img : 68
Img : 69
Img : 70
Img : 71
Img : 72
Img : 73
Img : 74
Img : 75
Img : 76
Img : 77
Img : 78
Img : 79
Img : 80
Img : 81
Img : 82
Img : 83
Img : 84
Img : 85
Img : 86
Img : 87
Img : 88
Img : 89
Img : 90
Img : 91
Img : 92
Img : 93
Img : 94
Img : 95
Img : 96
Img : 97
Img : 98
Img : 99
Img : 100
Img : 101
Img : 102
Img : 103
Img : 104
Img : 105
Img : 106
Img : 107
Img : 108
Img : 109
Img : 110
Img : 11

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in multiply


Img : 153


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in multiply


Img : 154


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in add


Img : 155
Img : 156
Img : 157
Img : 158
Img : 159
Img : 160
Img : 161
Img : 162
Img : 163
Img : 164
Img : 165
Img : 166
Img : 167
Img : 168
Img : 169
Img : 170
Img : 171
Img : 172
Img : 173
Img : 174
Img : 175
Img : 176
Img : 177
Img : 178
Img : 179
Img : 180
Img : 181
Img : 182
Img : 183
Img : 184
Img : 185
Img : 186
Img : 187
Img : 188
Img : 189
Img : 190
Img : 191
Img : 192
Img : 193
Img : 194
Img : 195
Img : 196
Img : 197
Img : 198
Img : 199
Img : 200
Img : 201
Img : 202
Img : 203
Img : 204
Img : 205
Img : 206
Img : 207
Img : 208
Img : 209
Img : 210
Img : 211
Img : 212
Img : 213
Img : 214
Img : 215
Img : 216
Img : 217
Img : 218
Img : 219
Img : 220
Img : 221
Img : 222
Img : 223
Img : 224
Img : 225
Img : 226
Img : 227
Img : 228
Img : 229
Img : 230
Img : 231
Img : 232
Img : 233
Img : 234
Img : 235
Img : 236
Img : 237
Img : 238
Img : 239
Img : 240
Img : 241
Img : 242
Img : 243
Img : 244
Img : 245
Img : 246
Img : 247
Img : 248
Img : 249
Img : 250
Img : 251
Img : 252
Img : 253
Img : 254


ValueError: unknown is not supported

In [20]:
vect_kdel(vCt)

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])